In [28]:
import os
os.chdir('..')

In [2]:
!pwd

/Users/chris/dsi/dsi_workspace/Capstone


In [23]:
import lib.download_mine as down
import lib.database_module as db
import tqdm
from rq import Queue
from redis import Redis
import lib.wiki_module as wiki
import lib.database_module as db
import lib.encoding_module as enc
from sklearn.externals import joblib
import unidecode
import rq_worker_download

In [9]:
q = Queue(connection=Redis(port=32379))
                

In [10]:
category_name = 'Mangroves'
category_dict = wiki.query_category(category_name)

In [11]:
for p in category_dict['pages']:
    print p['title']
#    wiki.query_page(pageid=str(p['pageid']))
    result = q.enqueue(fetch_and_store_page, str(p['pageid']) )

Mangrove
Mangrove swamp
Ecological values of mangroves
Mangrove tree distribution
Acanthus ebracteatus
Aegialitis
Aegiceras corniculatum
Alula Lagoon
Avicennia germinans
Avicennia marina
Avicennia officinalis
Barringtonia acutangula
Bruguiera
Bruguiera sexangula
Bu Tinah
Ceriops
Ceriops australis
Ceriops decandra
Ceriops tagal
Collier-Seminole State Park
Conocarpus erectus
False Divi Point
Graeme Hall Nature Sanctuary
Guapimirim Environmental Protection Area
Heritiera fomes
Heritiera littoralis
Hoi Ha Wan
Kandelia
Kandelia candel
Kandelia obovata
Kumarakom
Laguncularia racemosa
List of mangrove tree species of Australia
Lumnitzera
Madu Ganga
Mangrove restoration
Mangrove theme park, Kannur
Nypa fruticans
Palmyras Point
Pelliciera
Point Calimere
Point Calimere Wildlife and Bird Sanctuary
Rhizophora
Rhizophora apiculata
Rhizophora x lamarckii
Rhizophora mangle
Scyphiphora
Mangroves of the Straits of Malacca
Top dying disease
Tropical salt pond ecosystem
Umlalazi Nature Reserve
Xylocarpus

In [14]:
category_name = 'Mangroves'
category_dict = wiki.query_category(category_name)
db_location = 'local'
pickled_transformer = joblib.load('data/vectorizer.pkl')

In [15]:
category = down.wikipedia_get_page(category_name, category=True)
cate_id = category.keys()[0]
db.create_or_update_category_in_database(cate_id,category_name,db_location)

pages = down.wikipedia_get_pages_for_category(category_name)

Connected to server localhost.


In [11]:
for page in tqdm.tqdm(pages,desc='Downloading {}'.format(category_name)):
    result = q.enqueue_call(func=down.fetch_and_store_page_no_vector, args=("title="+page[u'title'],
                                                                   "page_id="+page['pageid'],
                                                                    "category_number="+str(cate_id),
                                                                    "db_location="+db_location))
    print page['pageid']



TypeError: cannot concatenate 'str' and 'int' objects

In [10]:
for page in tqdm.tqdm(pages[5:10],desc='Downloading {}'.format(category_name)):
    down.fetch_and_store_page_no_vector(title=page[u'title'],
                                        page_id=page['pageid'],
                                        category_number=cate_id,
                                        db_location=db_location)
    print page['pageid']

Connected to server localhost.
20971708


Connected to server localhost.
27417625


Connected to server localhost.
42665992


Connected to server localhost.
4372485


Connected to server localhost.
2983302


In [25]:
db_location

'local'

In [30]:
for page in tqdm.tqdm(pages,desc='Downloading {}'.format(category_name)):
    result = q.enqueue(rq_worker_download.fetch_and_store_page,  
                       page[u'title'], 
                       page['pageid'], 
                       cate_id, 
                       db_location)
    print page['pageid']

73448
3571585
26793225
26790549
23265805
20971708
27417625
42665992
4372485
2983302
20792317
11863897
29590712
29592807
24261777
22017682
53413878
46393577
39297858
3681742
1378146
28647136
8971349
16466539
31646244
11604378
914245
43761952
21716221
21715970
1329110
5904003
30591782
27897568
3505966
36309813
36808281
30863534
33722657
33890755
1252943
1252965
4372390
12912855
26244617
4372188
9321885
26817576
45490202
35300839
2853846
17726189
31247637
43495387
17726246


## rq docker file notes are here: 
http://sherzberg.github.io/docker-workshop/pages/full_example_python.html